# Programación rápida  con Julia (en serie)

Sí, este es un curso de computación paralela, pero para escribir programas paralelos eficientes, primero debemos aprender a escribir código serial Julia rápido. Esta es una introducción rápida a la programación de alto rendimiento (en serie).

_Recomiendo mucho_ revisar los Consejos de rendimiento en el manual [Performance Tips](https://docs.julialang.org/en/v1.1/manual/performance-tips/). Esto sólo va a introducir brevemente algunos de los conceptos principales.

## Medir, medir, medir.
Es muy fácil experimentar en Julia; puede probar rápidamente muchas opciones y ver cuál es la más rápida.

Usa el paquete  [BenchmarkTools](https://github.com/JuliaCI/BenchmarkTools.jl):

In [2]:
using BenchmarkTools

"""
    findclosest(data, point)

Un ejemplo simple que regresa el elemento en `data`que esta mas cercano al punto dado `point`.
"""
function findclosest(data, point)
    _, index =  findmin(abs.(data .- point))
    return data[index]
end
data = rand(5000)
findclosest(data, 0.5)

0.4999597704984099

In [3]:
#data = rand(5000)
findmin(abs.(data .- 0.5))

(4.022950159010552e-5, 1328)

In [4]:
@btime findclosest($data, $0.5)

  26.400 μs (2 allocations: 39.11 KiB)


0.4999597704984099

In [5]:
@benchmark findclosest($data, $0.5)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  24.000 μs …   8.763 ms  ┊ GC (min … max): 0.00% … 99.05%
 Time  (median):     73.200 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   79.453 μs ± 202.231 μs  ┊ GC (mean ± σ):  7.10% ±  2.79%

                                  ▄▄▅▆█▄▂                       
  ▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▂▆████████▇█▇▇▄▄▃▂▂▂▂▂▂▂▂▁▁▁▁▁ ▂
  24 μs           Histogram: frequency by time          105 μs <

 Memory estimate: 39.11 KiB, allocs estimate: 2.

### Profile!

In [6]:
using Profile

Profile.clear()
@profile for _ in 1:1000; findclosest(data, 0.5); end

Profile.print(maxdepth=11)

Overhead ╎ [+additional indent] Count File:Line; Function
  ╎52 @Base\task.jl:429; (::IJulia.var"#15#18")()
  ╎ 52 @IJulia\src\eventloop.jl:8; eventloop(socket::ZMQ.Socket)
  ╎  52 @Base\essentials.jl:714; invokelatest
  ╎   52 @Base\essentials.jl:716; #invokelatest#2
  ╎    52 ...c\execute_request.jl:67; execute_request(socket::ZMQ.Soc...
  ╎     52 ...\SoftGlobalScope.jl:65; softscope_include_string(m::Mod...
  ╎    ╎ 52 @Base\loading.jl:1196; include_string(mapexpr::typeo...
  ╎    ╎  52 @Base\boot.jl:373; eval
  ╎    ╎   52 ...ile\src\Profile.jl:28; top-level scope
  ╎    ╎    52 In[6]:4; macro expansion
  ╎    ╎     52 In[2]:9; findclosest(data::Vector{Flo...
  ╎    ╎    ╎ 40 @Base\broadcast.jl:860; materialize
  ╎    ╎    ╎ 12 @Base\reducedim.jl:1005; findmin
Total snapshots: 52


### Iterar!

Antes teniamos:
```julia
function findclosest(data, point)
    _, index =  findmin(abs.(data .- point))
    return data[index]
end
```

Propongamos una nueva definición que pueda combinar las dos operaciones:

In [7]:
function findclosest2(data, point)
    bestval = first(data)
    bestdist = abs(bestval - point)
    for elt in data
        dist = abs(elt - point)
        if dist < bestdist
            bestval = elt
            bestdist = dist
        end
    end
    return bestval
end

# Y se hace una verificación al azar para asegurar de que hicimos la optimización correctamente:
findclosest2(data, 0.5) == findclosest(data, 0.5)

true

In [8]:
@benchmark findclosest2($data, $0.5)

BenchmarkTools.Trial: 10000 samples with 6 evaluations.
 Range (min … max):   5.650 μs … 196.050 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     16.583 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   15.716 μs ±   5.100 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄▂▁                           █▂▄                            ▁
  ████▇▆▆▇█▆▇▆▇▇▆▆▅▆▆▆▆▅▅▆▆▇▆▅▆▇████▇▅▆▅▄▄▄▄▄▄▄▅▄▄▄▃▃▅▄▃▄▅▂▃▃▄ █
  5.65 μs       Histogram: log(frequency) by time      27.2 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [9]:
@btime findclosest2($data, $0.5)

  5.700 μs (0 allocations: 0 bytes)


0.4999597704984099

## Una palabra rápida en las macros

Las macros son esas cosas divertidas que comienzan con `@`. Pueden reinterpretar lo que
escribes y hacer algo diferente, esencialmente introduciendo una nueva palabra clave.

Por ejemplo, la macro `@assert` simplemente toma una expresión y lanza un
excepción si devuelve `falso`.

In [10]:
@assert 2+2 == 4

In [11]:
@assert 2+2 == 8

LoadError: AssertionError: 2 + 2 == 8

Lo hace literalmente reescribiendo lo que escribiste. Puedes verlo en acción
con `@macroexpand`

In [12]:
@macroexpand @assert 2+2 == 4

:(if 2 + 2 == 4
      nothing
  else
      Base.throw(Base.AssertionError("2 + 2 == 4"))
  end)

In [13]:
@macroexpand @time 2+2 == 4

quote
    #= timing.jl:216 =#
    while false
        #= timing.jl:216 =#
    end
    #= timing.jl:217 =#
    local var"#31#stats" = Base.gc_num()
    #= timing.jl:218 =#
    local var"#33#elapsedtime" = Base.time_ns()
    #= timing.jl:219 =#
    local var"#34#compile_elapsedtime" = Base.cumulative_compile_time_ns_before()
    #= timing.jl:220 =#
    local var"#32#val" = $(Expr(:tryfinally, :(2 + 2 == 4), quote
    var"#33#elapsedtime" = Base.time_ns() - var"#33#elapsedtime"
    #= timing.jl:222 =#
    var"#34#compile_elapsedtime" = Base.cumulative_compile_time_ns_after() - var"#34#compile_elapsedtime"
end))
    #= timing.jl:224 =#
    local var"#35#diff" = Base.GC_Diff(Base.gc_num(), var"#31#stats")
    #= timing.jl:225 =#
    Base.time_print(var"#33#elapsedtime", (var"#35#diff").allocd, (var"#35#diff").total_time, Base.gc_alloc_count(var"#35#diff"), var"#34#compile_elapsedtime", true)
    #= timing.jl:226 =#
    var"#32#val"
end

In [14]:
@which @time 2+2 == 4

var"@time"(__source__::LineNumberNode, __module__::Module, ex) in Base at timing.jl:214

Cada macro puede definir su propia sintaxis especial, y esto se usa ampliamente para la introspección de código, las mejoras de rendimiento en serie y, quizás lo más importante, ¡pprivilegios de paralelización!

## ¿Qué tan rápido es Julia?

Al comprender los conceptos básicos de cómo Julia _puede_ ser rápida, puede obtener una mejor
sentido de cómo escribir código Julia rápido.

Perhaps most importantly, Julia can reason about types. Recall: this is the definition of `findclosest2`:

```julia
function findclosest2(data, point)
    bestval = first(data)
    bestdist = abs(bestval - point)
    for elt in data
        dist = abs(elt - point)
        if dist < bestdist
            bestval = elt
            bestdist = dist
        end
    end
    return bestval
end
```

In [15]:
@code_typed optimize=false findclosest2(data, 0.5)

CodeInfo(
1 ─       (bestval = Main.first(data))::Float64
│   %2  = (bestval - point)::Float64
│         (bestdist = Main.abs(%2))::Float64
│   %4  = data::Vector{Float64}
│         (@_4 = Base.iterate(%4))::Union{Nothing, Tuple{Float64, Int64}}
│   %6  = (@_4 === nothing)::Bool
│   %7  = Base.not_int(%6)::Bool
└──       goto #6 if not %7
2 ┄ %9  = @_4::Tuple{Float64, Int64}
│         (elt = Core.getfield(%9, 1))::Float64
│   %11 = Core.getfield(%9, 2)::Int64
│   %12 = (elt - point)::Float64
│         (dist = Main.abs(%12))::Float64
│   %14 = (dist < bestdist)::Bool
└──       goto #4 if not %14
3 ─       (bestval = elt)::Float64
└──       (bestdist = dist)::Float64
4 ┄       (@_4 = Base.iterate(%4, %11))::Union{Nothing, Tuple{Float64, Int64}}
│   %19 = (@_4 === nothing)::Bool
│   %20 = Base.not_int(%19)::Bool
└──       goto #6 if not %20
5 ─       goto #2
6 ┄       return bestval
) => Float64

In [16]:
typeof(data)

Vector{Float64} (alias for Array{Float64, 1})

In [17]:
newdata = Real[data...]
typeof(newdata)

Vector{Real} (alias for Array{Real, 1})

In [19]:
@code_typed optimize=false findclosest2(newdata, 0.5)

CodeInfo(
1 ─       (bestval = Main.first(data))::Real
│   %2  = (bestval - point)::Any
│         (bestdist = Main.abs(%2))::Any
│   %4  = data::Vector{Real}
│         (@_4 = Base.iterate(%4))::Union{Nothing, Tuple{Real, Int64}}
│   %6  = (@_4 === nothing)::Bool
│   %7  = Base.not_int(%6)::Bool
└──       goto #6 if not %7
2 ┄ %9  = @_4::Tuple{Real, Int64}
│         (elt = Core.getfield(%9, 1))::Real
│   %11 = Core.getfield(%9, 2)::Int64
│   %12 = (elt - point)::Any
│         (dist = Main.abs(%12))::Any
│   %14 = (dist < bestdist)::Any
└──       goto #4 if not %14
3 ─       (bestval = elt)::Real
└──       (bestdist = dist)::Any
4 ┄       (@_4 = Base.iterate(%4, %11))::Union{Nothing, Tuple{Real, Int64}}
│   %19 = (@_4 === nothing)::Bool
│   %20 = Base.not_int(%19)::Bool
└──       goto #6 if not %20
5 ─       goto #2
6 ┄       return bestval
) => Real

In [20]:
@benchmark findclosest2($newdata, $0.5)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  142.000 μs …   3.003 ms  ┊ GC (min … max): 0.00% … 85.01%
 Time  (median):     403.900 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   433.027 μs ± 163.739 μs  ┊ GC (mean ± σ):  1.44% ±  4.75%

  ▂             ▁▂▇█▄▃▃▁▂▂▁                                     ▁
  █▇▆▆▆▆█████▇██████████████▇█▇▇█▆▆▇▆▆▆▆▇▆▆▆▆▆▆▅▆▇▇▆▅▄▅▄▅▅▅▅▄▅▅ █
  142 μs        Histogram: log(frequency) by time       1.07 ms <

 Memory estimate: 156.28 KiB, allocs estimate: 10002.

In [21]:
@code_warntype findclosest2(newdata, 0.5)

MethodInstance for findclosest2(::Vector{Real}, ::Float64)
  from findclosest2(data, point) in Main at In[7]:1
Arguments
  #self#::Core.Const(findclosest2)
  data::Vector{Real}
  point::Float64
Locals
  @_4::Union{Nothing, Tuple{Real, Int64}}
  bestdist::Any
  bestval::Real
  elt::Real
  dist::Any
Body::Real
1 ─       (bestval = Main.first(data))
│   %2  = (bestval - point)::Any
│         (bestdist = Main.abs(%2))
│   %4  = data::Vector{Real}
│         (@_4 = Base.iterate(%4))
│   %6  = (@_4 === nothing)::Bool
│   %7  = Base.not_int(%6)::Bool
└──       goto #6 if not %7
2 ┄ %9  = @_4::Tuple{Real, Int64}
│         (elt = Core.getfield(%9, 1))
│   %11 = Core.getfield(%9, 2)::Int64
│   %12 = (elt - point)::Any
│         (dist = Main.abs(%12))
│   %14 = (dist < bestdist)::Any
└──       goto #4 if not %14
3 ─       (bestval = elt)
└──       (bestdist = dist)
4 ┄       (@_4 = Base.iterate(%4, %11))
│   %19 = (@_4 === nothing)::Bool
│   %20 = Base.not_int(%19)::Bool
└──       goto #6 if not %

In [22]:
@code_warntype findclosest2(data, 0.5)

MethodInstance for findclosest2(::Vector{Float64}, ::Float64)
  from findclosest2(data, point) in Main at In[7]:1
Arguments
  #self#::Core.Const(findclosest2)
  data::Vector{Float64}
  point::Float64
Locals
  @_4::Union{Nothing, Tuple{Float64, Int64}}
  bestdist::Float64
  bestval::Float64
  elt::Float64
  dist::Float64
Body::Float64
1 ─       (bestval = Main.first(data))
│   %2  = (bestval - point)::Float64
│         (bestdist = Main.abs(%2))
│   %4  = data::Vector{Float64}
│         (@_4 = Base.iterate(%4))
│   %6  = (@_4 === nothing)::Bool
│   %7  = Base.not_int(%6)::Bool
└──       goto #6 if not %7
2 ┄ %9  = @_4::Tuple{Float64, Int64}
│         (elt = Core.getfield(%9, 1))
│   %11 = Core.getfield(%9, 2)::Int64
│   %12 = (elt - point)::Float64
│         (dist = Main.abs(%12))
│   %14 = (dist < bestdist)::Bool
└──       goto #4 if not %14
3 ─       (bestval = elt)
└──       (bestdist = dist)
4 ┄       (@_4 = Base.iterate(%4, %11))
│   %19 = (@_4 === nothing)::Bool
│   %20 = Base.not_

### Tipo de estabilidad

Una función se denomina de tipo estable si Julia puede inferir cuál será el tipo de salida basándose únicamente en los tipos de las entradas.

Cosas que frustran la estabilidad del tipo:

* Ejecutar cosas en el ámbito global: ¡cree funciones en su lugar!
* Contenedores de un tipo no concreto
* Estructuras con campos de tipo abstracto
* Globales no constantes (¡podrían cambiar!)
* Funciones que cambian lo que devuelven en función de los _valores_ :

#### Más sobre macros
Todas y cada una de las macros pueden definir su propia sintaxis. La macro `@benchmark` usa `$` de una manera especial. El objetivo detrás de `@benchmark` es evaluar el rendimiento de un fragmento de código como si estuviera escrito en una función. Use `$` para marcar lo que será un argumento o una variable local en la función. Olvidarse de usar `$` puede resultar en tiempos más rápidos o más lentos que el rendimiento del mundo real.

In [23]:
x = 0.5 # non-constant global
@btime sin(x)
@btime sin($x)

  18.908 ns (1 allocation: 16 bytes)
  3.103 ns (0 allocations: 0 bytes)


0.479425538604203

In [24]:
@btime sin(0.5) # constant literal!
@btime sin($0.5)

  0.900 ns (0 allocations: 0 bytes)
  3.700 ns (0 allocations: 0 bytes)


0.479425538604203

In [25]:
x=1
f()=sin(x)

f (generic function with 1 method)

In [26]:
@btime f()

  21.988 ns (1 allocation: 16 bytes)


0.8414709848078965

In [27]:
g(x)=sin(x)

g (generic function with 1 method)

In [28]:
@btime g(10.5)

  0.001 ns (0 allocations: 0 bytes)


-0.87969575997167

In [29]:
t=10.5
@btime g($t)

  6.300 ns (0 allocations: 0 bytes)


-0.87969575997167

## Especializaciones

El razonamiento de Julia sobre los tipos es particularmente importante ya que genera código de máquina especializado específicamente para los argumentos dados.

In [30]:
@code_llvm 1 + 2

;  @ int.jl:87 within `+`
; Function Attrs: uwtable
define i64 @"julia_+_3361"(i64 signext %0, i64 signext %1) #0 {
top:
  %2 = add i64 %1, %0
  ret i64 %2
}


Esto se aplica de la misma manera a cualquier función que escribamos, incluso a las más complicadas:

In [31]:
@code_llvm findclosest2(Float32[2.2,3.4,4.5],Float32(3.2))

;  @ In[7]:1 within `findclosest2`
; Function Attrs: uwtable
define float @julia_findclosest2_3382({}* nonnull align 16 dereferenceable(40) %0, float %1) #0 {
top:
;  @ In[7]:2 within `findclosest2`
; ┌ @ abstractarray.jl:398 within `first`
; │┌ @ array.jl:861 within `getindex`
    %2 = bitcast {}* %0 to { i8*, i64, i16, i16, i32 }*
    %3 = getelementptr inbounds { i8*, i64, i16, i16, i32 }, { i8*, i64, i16, i16, i32 }* %2, i64 0, i32 1
    %4 = load i64, i64* %3, align 8
    %.not = icmp eq i64 %4, 0
    br i1 %.not, label %oob, label %L22

L22:                                              ; preds = %top
    %5 = bitcast {}* %0 to float**
    %6 = load float*, float** %5, align 8
    %7 = load float, float* %6, align 4
; └└
;  @ In[7]:8 within `findclosest2`
; ┌ @ array.jl:835 within `iterate`
; │┌ @ int.jl:483 within `<` @ int.jl:476
    %.not1823.not = icmp eq i64 %4, 1
; │└
   br i1 %.not1823.not, label %L54, label %L48.preheader

L48.preheader:                                    

In [32]:
remove_comments(s) = join(filter(x->!startswith(x, ";"), split(s, "\n")), "\n")
sprint(code_llvm, findclosest2, Tuple{Vector{Float32}, Int}) |> remove_comments |> print

define float @julia_findclosest2_3390({}* nonnull align 16 dereferenceable(40) %0, i64 signext %1) #0 {
top:
    %2 = bitcast {}* %0 to { i8*, i64, i16, i16, i32 }*
    %3 = getelementptr inbounds { i8*, i64, i16, i16, i32 }, { i8*, i64, i16, i16, i32 }* %2, i64 0, i32 1
    %4 = load i64, i64* %3, align 8
    %.not = icmp eq i64 %4, 0
    br i1 %.not, label %oob, label %L23

L23:                                              ; preds = %top
    %5 = bitcast {}* %0 to float**
    %6 = load float*, float** %5, align 8
    %7 = load float, float* %6, align 4
       %8 = sitofp i64 %1 to float
    %.not1823.not = icmp eq i64 %4, 1
   br i1 %.not1823.not, label %L56, label %L50.preheader

L50.preheader:                                    ; preds = %L23
   %9 = fsub float %7, %8
   %10 = call float @llvm.fabs.f32(float %9)
   br label %L50

L50:                                              ; preds = %L50, %L50.preheader
   %11 = phi i64 [ %value_phi424, %L50 ], [ 1, %L50.preheader ]
   %value

## Efectos de hardware modernos

Hay muchas pequeñas peculiaridades de rendimiento en las computadoras modernas; Solo cubriré dos interesantes aquí:

In [33]:
@benchmark findclosest2($data, $0.5)

BenchmarkTools.Trial: 10000 samples with 4 evaluations.
 Range (min … max):   6.925 μs … 351.775 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     16.550 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   15.959 μs ±   5.341 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▂▂  ▁ ▂                                ▁ █▇▂ ▂▄             ▁
  ▆████████▇▇▇▇▆▇█▇▆▅▅▆▅▅▅▅▅▅▆▅▄▄▄▄▅▅▆▆▆▇██▇███▇███▇▇▆▆▅▅▄▅▃▅▄ █
  6.92 μs       Histogram: log(frequency) by time      20.4 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [34]:
sorteddata = sort(data)
@benchmark findclosest2($sorteddata, $0.5)

BenchmarkTools.Trial: 10000 samples with 4 evaluations.
 Range (min … max):   6.850 μs … 213.850 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     16.550 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   15.640 μs ±   4.504 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▂▂▁▂▁▂▂                             ▁▂█▇▂▁▄▃▁               ▁
  ▅██████████▇██▇▇▇▇██████▇▆▆▇▇▇███▇▇▇▇██████████▇▇▇▆▅▅▆▃▃▅▄▄▄ █
  6.85 μs       Histogram: log(frequency) by time      21.4 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

Desafortunadamente, esto no se puede demostrar en una plataforma de nube fortalecida... porque
es un gran riesgo de seguridad!

* https://meltdownattack.com
* https://discourse.julialang.org/t/psa-microbenchmarks-remember-branch-history/17436

In [35]:
idxs = sortperm(data)
sortedview = @view data[idxs]
@benchmark findclosest2($sortedview, $0.5)

BenchmarkTools.Trial: 10000 samples with 3 evaluations.
 Range (min … max):   7.100 μs … 136.567 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     16.133 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   15.367 μs ±   4.104 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

      ▁▁▁▂▁▁▁▁       ▁                █▃ ▂ ▃▁                  ▁
  █▇████████████▇█▇▇▇██▇▇▆▇▇▇▆▆▆▇▇▆▇▇▇█████████▆▇▆▆▄▅▅▅▃▃▃▅▄▄▄ █
  7.1 μs        Histogram: log(frequency) by time        22 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

### Latencias de memoria

| System Event                   | Actual Latency | Scaled Latency |
| ------------------------------ | -------------- | -------------- |
| One CPU cycle                  |     0.4 ns     |     1 s        |
| Level 1 cache access           |     0.9 ns     |     2 s        |
| Level 2 cache access           |     2.8 ns     |     7 s        |
| Level 3 cache access           |      28 ns     |     1 min      |
| Main memory access (DDR DIMM)  |    ~100 ns     |     4 min      |
| Intel Optane memory access     |     <10 μs     |     7 hrs      |
| NVMe SSD I/O                   |     ~25 μs     |    17 hrs      |
| SSD I/O                        |  50–150 μs     | 1.5–4 days     |
| Rotational disk I/O            |    1–10 ms     |   1–9 months   |
| Internet call: SF to NYC       |      65 ms     |     5 years    |
| Internet call: SF to Hong Kong |     141 ms     |    11 years    |

 (Obtenida de: https://www.prowesscorp.com/computer-latency-at-a-human-scale/)

### Conclusiones clave
* ¡Mide, mide, mide!
* Familiarícese con los consejos de rendimiento [Performance Tips](https://docs.julialang.org/en/v1/manual/performance-tips/)
* No tengas miedo de  `@code_typed`/`@code_warntype` y `@code_llvm`